# PROJET DE CRYPTANALYSE

## I Cryptanalyse de A5/2

In [1]:
def LFSR_step(state,f):
    L = len(state)
    out = state[0]
    state = state[1:]+[sum(f[j+1]*state[L-1-j] for j in range(L))]
    return out,state

### 1

In [2]:
def A5_2_init(K, IV, f1, f2, f3, f4):
    
    # On initialise les registres à 0.
    R1 = Sequence([GF(2)(0) for _ in range(19)]) 
    R2 = Sequence([GF(2)(0) for _ in range(22)])  
    R3 = Sequence([GF(2)(0) for _ in range(23)])
    R4 = Sequence([GF(2)(0) for _ in range(17)])
    
    
    for i in range(64):
        # On met à jour chaque registre avec son polynome
        # de rétroaction respectif.
        _, R1 = LFSR_step(R1, f1)
        _, R2 = LFSR_step(R2, f2)
        _, R3 = LFSR_step(R3, f3)
        _, R4 = LFSR_step(R4, f4)
        
       
        
        # On Xor pour chaque derniers bits de chaque registre 
        # avec l'indice i (qui est le nombre tours faits) 
        # de la clef K.
        R1[18] =  R1[18] + K[i]
        R2[21] =  R2[21] + K[i]
        R3[22] =  R3[22] + K[i]
        R4[16] =  R4[16] + K[i]
        
        
        
        
    for i in range(22):
        
        # On met à jour chaque registre avec son polynome
        # de rétroaction respectif.
        _, R1 = LFSR_step(R1, f1)
        _, R2 = LFSR_step(R2, f2)
        _, R3 = LFSR_step(R3, f3)
        _, R4 = LFSR_step(R4, f4)
        
        
        
        # On Xor pour chaque derniers bits de chaque registre
        # avec l'indice i du vecteur initial IV. 
        R1[18] =  R1[18] + IV[i]
        R2[21] =  R2[21] + IV[i]
        R3[22] =  R3[22] + IV[i]
        R4[16] =  R4[16] + IV[i]
        
        
    
    # On fixe certains bits des registres.
    R1[3] = 1
    R2[5] = 1
    R3[4] = 1
    R4[6] = 1
    
    
    
    return R1, R2, R3, R4    

In [3]:
def Maj(a, b, c):
    # 3 cas possibles pour la fonction majorité.
    m = 0 
    if a == b:
        m = a
        
    if a == c:
        m = a
        
    if b == c :
        m = b
        
    return m    

In [4]:
def A5_2_step(R1, R2, R3, R4, f1, f2, f3, f4):
    
    # On calcule la fonction majorité 
    # sur les indices 6, 13 et 9 de R4.
    m = Maj(R4[6], R4[13], R4[9])
    
    
    # On a 3 choix possibles.
    # Si à l'indice 6 de R4 est égale à m
    # on met à jour R1 en ignorant sa sortie.
    
    if R4[6] == m :
        _,  R1 = LFSR_step(R1, f1)
       
        
        
    # Si à l'indice 13 de R4 est égale à m
    # on met à jour R2 en ignorant sa sortie.
    if  R4[13] == m :
        _,  R2 = LFSR_step(R2, f2)
          
            
            
    # Si à l'indice 9 de R4 est égale à m 
    # on met à jour R3 en ignorant sa sortie.
    if  R4[9] == m :       
        _,  R3 = LFSR_step(R3, f3)
        
        
   
        
        
    # on met à jour R4 en ignorant la sortie.
    _, R4 = LFSR_step(R4, f4)   
    
    # On calcule y1, y2 et y3 avec les formules données.
    y1 = R1[0] + Maj(R1[3], R1[4] + 1, R1[6])
    y2 = R2[0] + Maj(R2[8], R2[5] + 1, R2[12])
    y3 = R3[0] + Maj(R3[4], R3[9] + 1, R3[6])
    
    # On ressort le résultat de l'additions des 3 variables
    return (y1 + y2 + y3),(R1, R2, R3, R4) 

In [5]:
def A5_2_production(R1, R2, R3, R4, f1, f2, f3, f4):
    
    # On exécute 99 fois la fonction "A5_2_step" en ignorant 
    # son bit de sortie 
    for i in range(99):
        _, (R1 ,R2 ,R3 ,R4) = A5_2_step(R1, R2, R3, R4, f1, f2, f3, f4)
    
    # On prend z une suite iniatilisé à une liste vide.
    z = []
    
    # On exécute 228 fois la fonction "A5_2_step" en utilisant son
    # bit de sortie.
    for i in range(228):
        zi, (R1 ,R2 ,R3 ,R4) = A5_2_step(R1, R2, R3, R4, f1, f2, f3, f4)
        z.append(zi)
        
    
    return z    

In [6]:
def A5_2(K, IV):
    
    # On crée les polynomes de rétroaction
    PR.<X> = PolynomialRing(GF(2))
    f1 = X**19 + X**18 + X**17 + X**14 + 1
    f2 = X**22 + X**21 + 1
    f3 = X**23 + X**22 + X**21 + X**8 + 1
    f4 = X**17 + X**12 + 1
    
    # Phase d'initialisation.
    R1, R2, R3, R4 = A5_2_init(K, IV, f1, f2, f3, f4)
    
    
    # Phase de productions.
    return A5_2_production(R1, R2, R3, R4, f1, f2, f3, f4)

In [7]:
# La clef K
K = Sequence([GF(2)(0), 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]);K

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [8]:
# IV
IV = Sequence([GF(2)(1), 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]);IV

[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [9]:
# Le résultat de la suite chiffrante
z = Sequence([GF(2)(1), 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]);z

[1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]

In [10]:
# On va tester si notre fonction nous donne les bonnes résultats
# prévues .
zprime = A5_2(K, IV) 

In [11]:
# On trouve qu'avec le K et IV donnée, on trouve bien le même z.
zprime == z

True

### 3

In [12]:
# On crée les matrices de rétroaction avec cette fonction:
def matf(f):# matrice de transition 
    n = f.degree()
    A = Matrix(GF(2),n,n)
    for i in range(n-1):
        A[i,i+1] = 1
    tmp = f.list()[1:] # c_1,c_2,...,c_L
    tmp.reverse()   # c_L,c_{L-1}, ... , c_1
    A[n-1] = vector(tmp)
    return A

In [13]:
def A5_2_stepret(R1, R2, R3, R4, f1, f2, f3, f4):
    #nos matrices de retroactions:
    A1 = matf(f1)
    A2 = matf(f2)
    A3 = matf(f3)
    A4 = matf(f4)
    
    
    # On calcule la fonction majorité 
    # sur les indices 6, 13 et 9 de R4.
    m = Maj(R4[6], R4[13], R4[9])
    
    
    # On a 3 choix possibles.
    # Si à l'indice 6 de R4 est égale à m
    # on met à jour R1 en ignorant sa sortie.
    
    if R4[6] == m :
        x1 =vector(R1)
        x1 = A1*x1
        R1 = Sequence(x1)
       
        
        
    # Si à l'indice 13 de R4 est égale à m
    # on met à jour R2 en ignorant sa sortie.
    if  R4[13] == m :
        x2 = vector(R2)
        x2 = A2*x2
        R2 = Sequence(x2)
          
            
            
    # Si à l'indice 9 de R4 est égale à m 
    # on met à jour R3 en ignorant sa sortie.
    if  R4[9] == m :       
        x3 = vector(R3)
        x3 = A3*x3
        R3 = Sequence(x3)
        
        
   
        
        
    # on met à jour R4 en ignorant la sortie.
    x4 = vector(R4)
    x4 = A4*x4
    R4 = Sequence(x4)   
    
    # On calcule y1, y2 et y3 avec les formules données.
    y1 = R1[0] + Maj(R1[3], R1[4] + 1, R1[6])
    y2 = R2[0] + Maj(R2[8], R2[5] + 1, R2[12])
    y3 = R3[0] + Maj(R3[4], R3[9] + 1, R3[6])
    
    # On ressort le résultat de l'additions des 3 variables
    return (y1 + y2 + y3),(R1, R2, R3, R4) 

In [14]:
# On suppose qu'on connait R4 et donc on va utiliser les K et IV données pour l'exemple du (1)

# La clef K
K = Sequence([GF(2)(0), 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# IV
IV = Sequence([GF(2)(1), 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# On utilise les mêmes polynômes de rétroactions que l'algorithme "A5_2_init"
PR.<X> = PolynomialRing(GF(2))
f1 = X**19 + X**18 + X**17 + X**14 + 1
f2 = X**22 + X**21 + 1
f3 = X**23 + X**22 + X**21 + X**8 + 1
f4 = X**17 + X**12 + 1
    
# On crée le R4 que l'on connait.
_, _, _, R4 = A5_2_init(K, IV, f1, f2, f3, f4)  


# Pour déclarer les 64 inconnues que l'on utilisera dans les registres R1, R2 et R3.
BPR = BooleanPolynomialRing(64, 'x')
v = BPR.gens()

# On crée les R1, R2 et R3 avec les inconnues associés.
R1 = Sequence([v[i] for i in range(19)])
R2 = Sequence([v[i] for i in range(19,41)])
R3 = Sequence([v[i] for i in range(41,64)])

# On exécute 99 fois la fonction "A5_2_step" en ignorant 
# son bit de sortie 
for i in range(99):
     _, (R1 ,R2 ,R3 ,R4) = A5_2_stepret(R1, R2, R3, R4, f1, f2, f3, f4)
    
# On exécute 228 fois la fonction "A5_2_step" en utilisant en ignorant
# bit de sortie.
for i in range(228):
    _, (R1 ,R2 ,R3 ,R4) = A5_2_stepret(R1, R2, R3, R4, f1, f2, f3, f4)

# On crée une liste d'équations linéaires.

Eql = [R1[i] for i in range(19)] + [R2[i] for i in range(22)] + [R3[i] for i in range(23)]
print("Taille de la liste d'équation =", len(Eql))

# On voit les 64 équations.
print("Les 64 équations sont : ")
for i in range(64):
    print("xprime%d = %s" %(i, Eql[i]))

Taille de la liste d'équation = 64
Les 64 équations sont : 
xprime0 = x4 + x8 + x10 + x11 + x12 + x13 + x15 + x17 + x18
xprime1 = x0 + x1 + x2 + x9 + x11 + x12 + x13 + x14 + x16 + x18
xprime2 = x0 + x3 + x5 + x10 + x12 + x13 + x14 + x15 + x17
xprime3 = x1 + x4 + x6 + x11 + x13 + x14 + x15 + x16 + x18
xprime4 = x0 + x1 + x7 + x12 + x14 + x15 + x16 + x17
xprime5 = x1 + x2 + x8 + x13 + x15 + x16 + x17 + x18
xprime6 = x0 + x1 + x3 + x5 + x9 + x14 + x16 + x17 + x18
xprime7 = x0 + x4 + x5 + x6 + x10 + x15 + x17 + x18
xprime8 = x0 + x2 + x6 + x7 + x11 + x16 + x18
xprime9 = x0 + x2 + x3 + x5 + x7 + x8 + x12 + x17
xprime10 = x1 + x3 + x4 + x6 + x8 + x9 + x13 + x18
xprime11 = x0 + x1 + x4 + x7 + x9 + x10 + x14
xprime12 = x1 + x2 + x5 + x8 + x10 + x11 + x15
xprime13 = x2 + x3 + x6 + x9 + x11 + x12 + x16
xprime14 = x3 + x4 + x7 + x10 + x12 + x13 + x17
xprime15 = x4 + x5 + x8 + x11 + x13 + x14 + x18
xprime16 = x0 + x1 + x2 + x6 + x9 + x12 + x14 + x15
xprime17 = x1 + x2 + x3 + x7 + x10 + x13 + x15 +

### 5

In [13]:
def Maj_al_n(a,b,c): # fonction majorité en forme algébrique normale.
    return a*b +b*c +a*c

In [14]:
def A5_2_stepv_2(R1, R2, R3, R4, f1, f2, f3, f4): 
    # On utilise la fonction majorité en forme algébrique normale.
    
    # On calcule la fonction majorité 
    # sur les indices 6, 13 et 9 de R4.
    m = Maj_al_n(R4[6], R4[13], R4[9])
    
    
    # On a 3 choix possibles.
    # Si à l'indice 6 de R4 est égale à m
    # on met à jour R1 en ignorant sa sortie.
    
    if R4[6] == m :
        _,  R1 = LFSR_step(R1, f1)
       
        
        
    # Si à l'indice 13 de R4 est égale à m
    # on met à jour R2 en ignorant sa sortie.
    if  R4[13] == m :
        _,  R2 = LFSR_step(R2, f2)
          
            
            
    # Si à l'indice 9 de R4 est égale à m 
    # on met à jour R3 en ignorant sa sortie.
    if  R4[9] == m :       
        _,  R3 = LFSR_step(R3, f3)
        
        
   
        
        
    # on met à jour R4 en ignorant la sortie.
    _, R4 = LFSR_step(R4, f4)   
    
    # On calcule y1, y2 et y3 avec les formules données.
    y1 = R1[0] +  Maj_al_n(R1[3], R1[4] + 1, R1[6])
    y2 = R2[0] +  Maj_al_n(R2[8], R2[5] + 1, R2[12])
    y3 = R3[0] +  Maj_al_n(R3[4], R3[9] + 1, R3[6])
    
    # On ressort le résultat de l'additions des 3 variables
    return (y1 + y2 + y3),(R1, R2, R3, R4) 

In [15]:
def A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4): # On utilise la fonction majorité en forme algébrique normale
    # Dans cette version on rajoute N pour pourvoir utiliser avec n'importe quel N.
    # On exécute 99 fois la fonction "A5_2_step" en ignorant 
    # son bit de sortie 
    for i in range(99):
        _, (R1 ,R2 ,R3 ,R4) = A5_2_stepv_2(R1, R2, R3, R4, f1, f2, f3, f4)
    
    # On prend z une suite iniatilisé à une liste vide.
    z = []
    
    # On exécute N fois la fonction "A5_2_step" en utilisant son
    # bit de sortie.
    for i in range(N):
        zi, (R1 ,R2 ,R3 ,R4) = A5_2_stepv_2(R1, R2, R3, R4, f1, f2, f3, f4)
        z.append(zi)
        
    
    return z    

In [16]:
def A5_2v_2(N,K, IV):
    
    # On crée les polynomes de rétroaction
    PR.<X> = PolynomialRing(GF(2))
    f1 = X**19 + X**18 + X**17 + X**14 + 1
    f2 = X**22 + X**21 + 1
    f3 = X**23 + X**22 + X**21 + X**8 + 1
    f4 = X**17 + X**12 + 1
    
    # Phase d'initialisation.
    R1, R2, R3, R4 = A5_2_init(K, IV, f1, f2, f3, f4)
    
    
    # Phase de productions.
    return A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4)

In [17]:
# On suppose qu'on connait R4 et donc on va utiliser les K et IV données pour l'exemple du (1)

# La clef K
K = Sequence([GF(2)(0), 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# IV
IV = Sequence([GF(2)(1), 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# On utilise les mêmes polynômes de rétroactions que l'algorithme "A5_2_init"
PR.<X> = PolynomialRing(GF(2))
f1 = X**19 + X**18 + X**17 + X**14 + 1
f2 = X**22 + X**21 + 1
f3 = X**23 + X**22 + X**21 + X**8 + 1
f4 = X**17 + X**12 + 1
    
# On crée le R4 que l'on connait.
_, _, _, R4 = A5_2_init(K, IV, f1, f2, f3, f4)  


# Pour déclarer les 64 inconnues que l'on utilisera dans les registres R1, R2 et R3.
BPR = BooleanPolynomialRing(64, 'x')
v = BPR.gens()

# On crée les R1, R2 et R3 avec les inconnues associés.
R1 = Sequence([v[i] for i in range(19)])
R2 = Sequence([v[i] for i in range(19,41)])
R3 = Sequence([v[i] for i in range(41,64)])

# On veut N =228
N = 228

# On calcule la suite chiffré z avec ses 64 inconnues.
zprime = A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4)

# On a crée une liste d'équations quadratiques et on l'affiche les 228 équations.
print("Taille de la liste d'équation =", len(zprime))

# On voit les 228 équations.
print("Les %d équations sont : " %(N))
for i in range(228):
    print("zprime[%d] = %s" %(i, zprime[i]))

Taille de la liste d'équation = 228
Les 228 équations sont : 
zprime[0] = x3 + x4 + x5 + x6*x7 + x6*x8 + x6*x12 + x6*x17 + x7*x11 + x7*x12 + x7*x13 + x7*x14 + x7*x15 + x7 + x8*x11 + x8*x12 + x8*x13 + x8*x14 + x8*x15 + x10 + x11*x12 + x11*x17 + x11 + x12*x13 + x12*x14 + x12*x15 + x12*x17 + x12 + x13*x17 + x14*x17 + x14 + x15*x17 + x17 + x23 + x24 + x25 + x26 + x28*x31 + x28*x32 + x28*x33 + x28*x34 + x28*x35 + x28*x36 + x28*x37 + x28*x38 + x29*x31 + x29*x32 + x29*x33 + x29*x34 + x29*x35 + x29*x36 + x29*x37 + x29*x38 + x30*x31 + x30*x32 + x30*x33 + x30*x34 + x30*x35 + x30*x36 + x30*x37 + x30*x38 + x31*x32 + x31*x33 + x31*x34 + x32*x35 + x32*x36 + x32*x37 + x32*x38 + x32 + x33*x35 + x33*x36 + x33*x37 + x33*x38 + x33 + x34*x35 + x34*x36 + x34*x37 + x34*x38 + x34 + x35 + x36 + x37 + x38 + x41 + x43*x45 + x43*x47 + x43*x52 + x43*x60 + x43*x61 + x43*x63 + x44*x45 + x44*x47 + x44*x52 + x44*x60 + x44*x61 + x44*x63 + x45*x50 + x45*x52 + x45*x54 + x45*x56 + x45*x57 + x45*x59 + x45*x63 + x47*x50 + 

### 6

In [18]:
# Pour déclarer les 64 inconnues que l'on utilisera dans les registres R1, R2 et R3.
BPR = BooleanPolynomialRing(64, 'x')
v = BPR.gens()

# On sait x3 = x24 = x45 = 1, sont les variables connues, on crée M1 la liste des monômes
# de degré 1 sans ses 3 variables.
M1 = [v[i] for i in range(3)]+[v[i] for i in range(4,24)]+[v[i] for i in range(25,45)]+[v[i] for i in range(46,64)]
len(M1)

# On crée M la liste de tous les monômes possibles.
M = [M1[i] for i in range(61)]+[M1[i]*M1[j] for i in range(18) for j in range(i+1,18)]+[M1[i]*M1[j] for i in range(18,39) for j in range(i+1,39)]+[M1[i]*M1[j] for i in range(39,61) for j in range(i+1,61)]
print("M =", M)
print("La longueur de M est :", len(M))
# On voit que sa longueur est bien 655
N = 228
L = 655

M = [x0, x1, x2, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, x21, x22, x23, x25, x26, x27, x28, x29, x30, x31, x32, x33, x34, x35, x36, x37, x38, x39, x40, x41, x42, x43, x44, x46, x47, x48, x49, x50, x51, x52, x53, x54, x55, x56, x57, x58, x59, x60, x61, x62, x63, x0*x1, x0*x2, x0*x4, x0*x5, x0*x6, x0*x7, x0*x8, x0*x9, x0*x10, x0*x11, x0*x12, x0*x13, x0*x14, x0*x15, x0*x16, x0*x17, x0*x18, x1*x2, x1*x4, x1*x5, x1*x6, x1*x7, x1*x8, x1*x9, x1*x10, x1*x11, x1*x12, x1*x13, x1*x14, x1*x15, x1*x16, x1*x17, x1*x18, x2*x4, x2*x5, x2*x6, x2*x7, x2*x8, x2*x9, x2*x10, x2*x11, x2*x12, x2*x13, x2*x14, x2*x15, x2*x16, x2*x17, x2*x18, x4*x5, x4*x6, x4*x7, x4*x8, x4*x9, x4*x10, x4*x11, x4*x12, x4*x13, x4*x14, x4*x15, x4*x16, x4*x17, x4*x18, x5*x6, x5*x7, x5*x8, x5*x9, x5*x10, x5*x11, x5*x12, x5*x13, x5*x14, x5*x15, x5*x16, x5*x17, x5*x18, x6*x7, x6*x8, x6*x9, x6*x10, x6*x11, x6*x12, x6*x13, x6*x14, x6*x15, x6*x16, x6*x17, x6*x18, x7*x8, x7*x9, x7*x10, x7*x11, x7*x12

### 7

In [20]:
# On suppose qu'on connait R4 et donc on va utiliser les K et IV données pour l'exemple du (1)

# La clef K
K = Sequence([GF(2)(0), 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# IV
IV = Sequence([GF(2)(1), 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# On utilise les mêmes polynômes de rétroactions que l'algorithme "A5_2_init"
PR.<X> = PolynomialRing(GF(2))
f1 = X**19 + X**18 + X**17 + X**14 + 1
f2 = X**22 + X**21 + 1
f3 = X**23 + X**22 + X**21 + X**8 + 1
f4 = X**17 + X**12 + 1
    
# On crée le R4 que l'on connait.
_, _, _, R4 = A5_2_init(K, IV, f1, f2, f3, f4)  


# Pour déclarer les 64 inconnues que l'on utilisera dans les registres R1, R2 et R3.
BPR = BooleanPolynomialRing(64, 'x')
v = BPR.gens()

# On crée les R1, R2 et R3 avec les inconnues associés et leurs bits connus.
R1 = Sequence([v[i] for i in range(3)]+[1]+[v[i] for i in range(4,19)])
R2 = Sequence([v[i] for i in range(19,24)]+[1]+[v[i] for i in range(25,41)])
R3 = Sequence([v[i] for i in range(41,45)]+[1]+[v[i] for i in range(46,64)])

# On calcule la suite chiffré z avec ses 64 inconnues.
zprimev2 = A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4)

# On a crée une liste d'équations quadratiques et on l'affiche les 228 équations.
print("Taille de la liste d'équation =", len(zprimev2))

# On voit les 228 équations.
print("Les %d équations sont : " %(N))
for i in range(228):
    print("zprime%d = %s" %(i, zprimev2[i]))

Taille de la liste d'équation = 228
Les 228 équations sont : 
zprime0 = x4 + x5 + x6*x7 + x6*x8 + x6*x12 + x6*x17 + x7*x11 + x7*x12 + x7*x13 + x7*x14 + x7*x15 + x7 + x8*x11 + x8*x12 + x8*x13 + x8*x14 + x8*x15 + x10 + x11*x12 + x11*x17 + x11 + x12*x13 + x12*x14 + x12*x15 + x12*x17 + x12 + x13*x17 + x14*x17 + x14 + x15*x17 + x17 + x23 + x25 + x26 + x28*x31 + x28*x32 + x28*x33 + x28*x34 + x28*x35 + x28*x36 + x28*x37 + x28*x38 + x29*x31 + x29*x32 + x29*x33 + x29*x34 + x29*x35 + x29*x36 + x29*x37 + x29*x38 + x30*x31 + x30*x32 + x30*x33 + x30*x34 + x30*x35 + x30*x36 + x30*x37 + x30*x38 + x31*x32 + x31*x33 + x31*x34 + x32*x35 + x32*x36 + x32*x37 + x32*x38 + x32 + x33*x35 + x33*x36 + x33*x37 + x33*x38 + x33 + x34*x35 + x34*x36 + x34*x37 + x34*x38 + x34 + x35 + x36 + x37 + x38 + x41 + x43*x47 + x43*x52 + x43*x60 + x43*x61 + x43*x63 + x43 + x44*x47 + x44*x52 + x44*x60 + x44*x61 + x44*x63 + x44 + x47*x50 + x47*x52 + x47*x54 + x47*x56 + x47*x57 + x47*x59 + x47*x63 + x47 + x48 + x50*x52 + x50*x60 +

In [21]:
# On crée la matrice Mat de taille N*L dans F2.
Mat = matrix(GF(2),N,L)

# On crée le vecteur de longueur N.
Vec = vector(GF(2),N)

# On utilise le zprime calculé dans le (5).
# On utilise les indications de l'énoncé.
for i in range(N):
    Vec[i] = zprimev2[i].monomial_coefficient(1)
    for j in range(L):
        Mat[i,j] = zprimev2[i].monomial_coefficient(M[j])
        
print("Vec =", Vec)
print("Mat =", Mat) 
print("\n")

Vec = (0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1)
Mat = [0 0 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 0 

### 8

In [22]:
N = 700
# On considère une exécution de A5/2 donnant N = 700 bits de suite chiffrante avec  
IV = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
# Valeur de R4 après la phase d'initialisation
R4 =  Sequence([GF(2)(1), 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1])
# Les 700 bits de suite chiffrante
z700 = Sequence([GF(2)(0), 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])

In [23]:
 
# Pour déclarer les 64 inconnues que l'on utilisera dans les registres R1, R2 et R3.
BPR = BooleanPolynomialRing(64, 'x')
v = BPR.gens()

# On utilise les mêmes polynômes de rétroactions que l'algorithme "A5_2_init"
PR.<X> = PolynomialRing(GF(2))
f1 = X**19 + X**18 + X**17 + X**14 + 1
f2 = X**22 + X**21 + 1
f3 = X**23 + X**22 + X**21 + X**8 + 1
f4 = X**17 + X**12 + 1

# On crée les R1, R2 et R3 avec les inconnues associés et leurs bits connus.
R1 = Sequence([v[i] for i in range(3)]+[1]+[v[i] for i in range(4,19)])
R2 = Sequence([v[i] for i in range(19,24)]+[1]+[v[i] for i in range(25,41)])
R3 = Sequence([v[i] for i in range(41,45)]+[1]+[v[i] for i in range(46,64)])

# On calcule la suite chiffré z avec ses 64 inconnues.
zprime700 = A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4)

# On a crée une liste d'équations quadratiques et on l'affiche les 700 équations.
print("Taille de la liste d'équation =", len(zprime700))

# On voit les 700 équations.
print("Les %d équations sont : " %(N))
for i in range(N):
    print("zprime700[%d] = %s" %(i, zprime700[i]))
    

Taille de la liste d'équation = 700
Les 700 équations sont : 
zprime700[0] = x1*x9 + x1*x14 + x1*x15 + x1*x16 + x1*x17 + x1*x18 + x1 + x2*x9 + x2*x14 + x2*x15 + x2*x16 + x2*x17 + x2*x18 + x2 + x6*x9 + x6*x14 + x6*x15 + x6*x16 + x6*x17 + x6*x18 + x7 + x8 + x9*x10 + x9*x11 + x9*x15 + x9 + x10*x14 + x10*x15 + x10*x16 + x10*x17 + x10*x18 + x10 + x11*x14 + x11*x15 + x11*x16 + x11*x17 + x11*x18 + x13 + x14*x15 + x15*x16 + x15*x17 + x15*x18 + x16 + x18 + x19 + x20 + x22*x25 + x22*x26 + x22*x27 + x22*x28 + x22*x29 + x22*x30 + x22*x31 + x22*x32 + x23*x25 + x23*x26 + x23*x27 + x23*x28 + x23*x29 + x23*x30 + x23*x31 + x23*x32 + x25*x26 + x25*x27 + x25*x28 + x25 + x26*x29 + x26*x30 + x26*x31 + x26*x32 + x27*x29 + x27*x30 + x27*x31 + x27*x32 + x28*x29 + x28*x30 + x28*x31 + x28*x32 + x39 + x41*x42 + x41*x46 + x41*x47 + x41*x50 + x41*x52 + x41*x58 + x41*x60 + x42*x43 + x42*x48 + x42*x49 + x42*x50 + x42*x52 + x42*x55 + x42*x56 + x42*x57 + x42*x58 + x42*x59 + x42*x61 + x42*x62 + x42 + x43*x46 + x43*x47 

In [24]:
# On crée la matrice Mat de taille N*L dans F2.
Mat700 = matrix(GF(2),N,L)

# On crée le vecteur de longueur N.
Vec700 = vector(GF(2),N)

# On utilise le zprime700 .
# On utilise les indications de l'énoncé.
for i in range(N):
    Vec700[i] = zprime700[i].monomial_coefficient(1)
    for j in range(L):
        Mat700[i,j] = zprime700[i].monomial_coefficient(M[j])
        
print("Vec700 =", Vec700)
print("\n")
print("Mat700 =", Mat700)

Vec700 = (0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 

In [26]:
# Pour résoudre l'équation linéaire, on va utiliser la fonction solve_right() donnée dans l'énoncé.
# On additionne vec700 à z700.
Right = [Vec700[i] + z700[i] for i in range(N)]
#on résoud l'équation grâce à la fonction, en mettant tous les inconnues à gauche de l'égalité
# et le reste à droite, on pose sol700 le résultat de la fonction:
Sol700 = Mat700.solve_right(Right)
Sol700

(1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [27]:
#On a besoin que les 61 premiers bits du sol700 pour récupérer 
# R1, R2 et R3 :
R1 = Sequence([Sol700[i] for i in range(3)]+[1]+[Sol700[i] for i in range(3,18)])
R2 = Sequence([Sol700[i] for i in range(18,23)]+[1]+[Sol700[i] for i in range(23,39)])
R3 = Sequence([Sol700[i] for i in range(39,43)]+[1]+[Sol700[i] for i in range(43,61)])
print("R1 =", R1)
print("R2 =", R2)
print("R3 =", R3)
print("R4 =", R4)

R1 = [1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]
R2 = [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1]
R3 = [1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
R4 = [1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1]


In [28]:
# On teste si ça nous donne le bon résultat:
ztest700 = A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4)
ztest700 == z700

True

### 10

In [29]:
def matf(f):# matrice de transition 
    n = f.degree()
    A = Matrix(GF(2),n,n)
    for i in range(n-1):
        A[i,i+1] = 1
    tmp = f.list()[1:] # c_1,c_2,...,c_L
    tmp.reverse()   # c_L,c_{L-1}, ... , c_1
    A[n-1] = vector(tmp)
    return A

In [30]:
# On utilise les mêmes polynômes de rétroactions que l'algorithme "A5_2_init"
PR.<X> = PolynomialRing(GF(2))
f1 = X**19 + X**18 + X**17 + X**14 + 1
f2 = X**22 + X**21 + 1
f3 = X**23 + X**22 + X**21 + X**8 + 1
f4 = X**17 + X**12 + 1

In [31]:
# On construit les matrices de retroactions pour chaque fonction de rétroaction
A1 = matf(f1)
A2 = matf(f2)
A3 = matf(f3)
A4 = matf(f4)

In [32]:
# Pour déclarer les 64 inconnues que l'on utilisera dans la clef K.
BPR = BooleanPolynomialRing(64, 'K')
v = BPR.gens()

# Comme dans l'algorithme de A5/2-init les registres sont vides des 4 LFSR sont nulles.
X1 = vector([0 for _ in range(19)]) # registre du LFSR1
X2 = vector([0 for _ in range(22)]) # registre du LFSR2
X3 = vector([0 for _ in range(23)]) # registre du LFSR3
X4 = vector([0 for _ in range(17)]) # registre du LFSR4

# On calcule les registres comme dans le résultat du (9).
# Dans la première boucle for:
for i in range(64):
    X1 = A1*X1+vector([0 for _ in range(18)]+[v[i]])
    X2 = A2*X2+vector([0 for _ in range(21)]+[v[i]])
    X3 = A3*X3+vector([0 for _ in range(22)]+[v[i]])
    X4 = A4*X4+vector([0 for _ in range(16)]+[v[i]])
    
# Comme IV est nulle, dans la deuxième boucle for on multiplie pour chaque registre par leurs matrices 
# de rétroaction qui sont mis à la puissance 22.
X1 = A1**22*X1
X2 = A2**22*X2
X3 = A3**22*X3
X4 = A4**22*X4

# A la fin de cet algorithme, on met un seul bit à 1 dans chaque registre :
X1[3] = 1
X2[5] = 1
X3[4] = 1
X4[6] = 1           

In [33]:
# On fait une liste équation
L = list(X1)+list(X2)+list(X3)+list(X4)

In [34]:
# On pose l = len(L)
l =len(L)

In [35]:
# On concatène les registres trouvés dans le (8)
R = list(R1)+list(R2)+list(R3)+list(R4)

In [36]:
# On va calculer l'idéal:
I  = ideal([L[i] + R[i] for i in range(81)]);I  

Ideal (K10 + K13 + K16 + K17 + K19 + K20 + K21 + K22 + K25 + K29 + K31 + K33 + K39 + K48 + K49 + K50 + K53 + 1, K0 + K11 + K14 + K17 + K18 + K20 + K21 + K22 + K23 + K26 + K30 + K32 + K34 + K40 + K49 + K50 + K51 + K54, K1 + K12 + K15 + K18 + K19 + K21 + K22 + K23 + K24 + K27 + K31 + K33 + K35 + K41 + K50 + K51 + K52 + K55 + 1, 0, K1 + K3 + K14 + K17 + K20 + K21 + K23 + K24 + K25 + K26 + K29 + K33 + K35 + K37 + K43 + K52 + K53 + K54 + K57 + 1, K0 + K2 + K4 + K15 + K18 + K21 + K22 + K24 + K25 + K26 + K27 + K30 + K34 + K36 + K38 + K44 + K53 + K54 + K55 + K58, K0 + K1 + K3 + K5 + K16 + K19 + K22 + K23 + K25 + K26 + K27 + K28 + K31 + K35 + K37 + K39 + K45 + K54 + K55 + K56 + K59, K0 + K1 + K2 + K4 + K6 + K17 + K20 + K23 + K24 + K26 + K27 + K28 + K29 + K32 + K36 + K38 + K40 + K46 + K55 + K56 + K57 + K60, K0 + K1 + K2 + K3 + K5 + K7 + K18 + K21 + K24 + K25 + K27 + K28 + K29 + K30 + K33 + K37 + K39 + K41 + K47 + K56 + K57 + K58 + K61 + 1, K0 + K1 + K2 + K3 + K4 + K6 + K8 + K19 + K22 + K25 + K26

In [37]:
# On utilise la base groebner.
Iq = I.groebner_basis();list(Iq)


[K0,
 K1 + 1,
 K2,
 K3 + 1,
 K4,
 K5,
 K6 + 1,
 K7,
 K8,
 K9 + 1,
 K10,
 K11,
 K12 + 1,
 K13,
 K14,
 K15 + 1,
 K16 + 1,
 K17,
 K18,
 K19,
 K20,
 K21,
 K22,
 K23,
 K24 + 1,
 K25,
 K26 + 1,
 K27,
 K28 + 1,
 K29 + 1,
 K30 + 1,
 K31 + 1,
 K32 + 1,
 K33 + 1,
 K34 + 1,
 K35 + 1,
 K36 + 1,
 K37,
 K38 + 1,
 K39 + 1,
 K40,
 K41 + 1,
 K42 + 1,
 K43 + 1,
 K44 + 1,
 K45 + 1,
 K46,
 K47,
 K48,
 K49,
 K50,
 K51,
 K52 + 1,
 K53,
 K54,
 K55,
 K56,
 K57,
 K58,
 K59 + 1,
 K60 + 1,
 K61,
 K62,
 K63]

In [38]:
# On cherche les coefficients qui additionnent les variables pour avoir une clef Kprime.
Kprime = [el.constant_coefficient() for el in Iq];Kprime

[0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0]

In [39]:
#On calcule avec Kprime pour chercher les Riprime avec 1<=i<=4
R1prime, R2prime, R3prime,R4prime =A5_2_init(Kprime, IV, f1, f2, f3, f4)

# On teste si ça nous donne les bons résultats:
print(R1prime == R1)
print(R2prime == R2)
print(R3prime == R3)
print(R4prime == R4)

True
True
True
True


### On voit qu'on a la bonne clef.

### 13

In [39]:
load('A5_2-3frames.sage')

In [40]:
IV= Sequence([GF(2)(0), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) # pour z0
IVprime = Sequence([GF(2)(0), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]) # pour z1
IVtilde = Sequence([GF(2)(0), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]) # pour z2

In [41]:
# On pose les registres des LFSRi Ri pour z0, Riprime pour z1 et Ritilde pour z2 avec 1 <= i <= 4
# Pour déclarer les 64 inconnues que l'on utilisera dans les registres R1, R2, R3, R1prime, R2prime, R3prime
# R1tilde, R2tilde et R3tilde
BPR = BooleanPolynomialRing(64, 'x')
v = BPR.gens()

# On crée les R1, R2 et R3 avec les inconnues associés et leurs bits connus.
R1 = Sequence([v[i] for i in range(3)]+[1]+[v[i] for i in range(4,19)])
R2 = Sequence([v[i] for i in range(19,24)]+[1]+[v[i] for i in range(25,41)])
R3 = Sequence([v[i] for i in range(41,45)]+[1]+[v[i] for i in range(46,64)])

print("R1 =",R1)
print("R2 =",R2)
print("R3 =",R3)
print("R4 =",R4)

R1 = [x0, x1, x2, 1, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18]
R2 = Polynomial Sequence with 22 Polynomials in 21 Variables
R3 = Polynomial Sequence with 23 Polynomials in 22 Variables
R4 = [0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1]


In [42]:
def matf(f):# matrice de transition 
    n = f.degree()
    A = Matrix(GF(2),n,n)
    for i in range(n-1):
        A[i,i+1] = 1
    tmp = f.list()[1:] # c_1,c_2,...,c_L
    tmp.reverse()   # c_L,c_{L-1}, ... , c_1
    A[n-1] = vector(tmp)
    return A

In [43]:
# On utilise les mêmes polynômes de rétroactions que l'algorithme "A5_2_init"
PR.<X> = PolynomialRing(GF(2))
f1 = X**19 + X**18 + X**17 + X**14 + 1
f2 = X**22 + X**21 + 1
f3 = X**23 + X**22 + X**21 + X**8 + 1
f4 = X**17 + X**12 + 1

In [44]:
# On construit les matrices de retroactions pour chaque fonction de rétroaction
A1 = matf(f1)
A2 = matf(f2)
A3 = matf(f3)
A4 = matf(f4)

In [45]:
# Pour faire les calculs je transforme les registres en vecteur.
X1 = vector(R1)
X2 = vector(R2)
X3 = vector(R3)
X4 = vector(R4)

In [46]:
# On calcul les registres des autres suites chiffrantes.
# On commence par celle de z1:
X1prime = X1 + vector([0 for _ in range(18)]+[IVprime[21]])
X2prime = X2 + vector([0 for _ in range(21)]+[IVprime[21]])
X3prime = X3 + vector([0 for _ in range(22)]+[IVprime[21]])
X4prime = X4 + vector([0 for _ in range(16)]+[IVprime[21]])

# on met un seul bit à 1 dans chaque registre :
X1prime[3] = 1
X2prime[5] = 1
X3prime[4] = 1
X4prime[6] = 1 

# On le remet en sequence
R1prime = Sequence(X1prime)
R2prime = Sequence(X2prime)
R3prime = Sequence(X3prime)
R4prime = Sequence(X4prime)

print("R1prime =",R1prime)
print("R2prime =",R2prime)
print("R3prime =",R3prime)
print("R4prime =",R4prime)

R1prime = [x0, x1, x2, 1, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18 + 1]
R2prime = Polynomial Sequence with 22 Polynomials in 21 Variables
R3prime = Polynomial Sequence with 23 Polynomials in 22 Variables
R4prime = [0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0]


In [47]:
# et celle de z2:
X1tilde = X1 + A1 * vector([0 for _ in range(18)]+[IVtilde[20]])
X2tilde = X2 + A2 * vector([0 for _ in range(21)]+[IVtilde[20]])
X3tilde = X3 + A3 * vector([0 for _ in range(22)]+[IVtilde[20]])
X4tilde = X4 + A4 * vector([0 for _ in range(16)]+[IVtilde[20]])

# on met un seul bit à 1 dans chaque registre :
X1tilde[3] = 1
X2tilde[5] = 1
X3tilde[4] = 1
X4tilde[6] = 1 

# On le remet en sequence
R1tilde = Sequence(X1tilde)
R2tilde = Sequence(X2tilde)
R3tilde = Sequence(X3tilde)
R4tilde = Sequence(X4tilde)

print("R1tilde =",R1tilde)
print("R2tilde =",R2tilde)
print("R3tilde =",R3tilde)
print("R4tilde =",R4tilde)

R1tilde = [x0, x1, x2, 1, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17 + 1, x18]
R2tilde = Polynomial Sequence with 22 Polynomials in 21 Variables
R3tilde = Polynomial Sequence with 23 Polynomials in 22 Variables
R4tilde = [0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1]


In [48]:
# On pose N = 228
N = 228
# On calcule les z théoriques 
z = A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4)
zprime = A5_2_productionv_2(N, R1prime, R2prime, R3prime, R4prime, f1, f2, f3, f4)
ztilde = A5_2_productionv_2(N, R1tilde, R2tilde, R3tilde, R4tilde, f1, f2, f3, f4)

In [49]:
# On crée une liste d'équations
Eq =z+zprime+ztilde

# On crée une liste de résultats voulues
Res = z0+z1+z2

In [50]:
# On sait x3 = x24 = x45 = 1, sont les variables connues, on crée M1 la liste des monômes
# de degré 1 sans ses 3 variables.
M1 = [v[i] for i in range(3)]+[v[i] for i in range(4,24)]+[v[i] for i in range(25,45)]+[v[i] for i in range(46,64)]

# On crée M la liste de tous les monômes possibles.
M = [M1[i] for i in range(61)]+[M1[i]*M1[j] for i in range(18) for j in range(i+1,18)]+[M1[i]*M1[j] for i in range(18,39) for j in range(i+1,39)]+[M1[i]*M1[j] for i in range(39,61) for j in range(i+1,61)]

L = len(M)

In [52]:
# On crée la matrice Mat de taille N*L dans F2.
Mat = matrix(GF(2),3*N,L)

# On crée le vecteur de longueur N.
Vec = vector(GF(2),3*N)


# On utilise les indications de l'énoncé.
for i in range(3*N):
    Vec[i] = Eq[i].monomial_coefficient(1)
    for j in range(L):
        Mat[i,j] = Eq[i].monomial_coefficient(M[j])


In [53]:
# On rajoute le Vecteur aux résultats prévues
Right = [Vec[i]+Res[i] for i in range(3*N)]   

In [54]:
Sol = Mat.solve_right(Right)

In [55]:
#On a besoin que les 61 premiers bits du sol pour récupérer 
# R1, R2 et R3 :
R1 = Sequence([Sol[i] for i in range(3)]+[1]+[Sol[i] for i in range(3,18)])
R2 = Sequence([Sol[i] for i in range(18,23)]+[1]+[Sol[i] for i in range(23,39)])
R3 = Sequence([Sol[i] for i in range(39,43)]+[1]+[Sol[i] for i in range(43,61)])
print("R1 =", R1)
print("R2 =", R2)
print("R3 =", R3)
print("R4 =", R4)

R1 = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0]
R2 = [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0]
R3 = [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
R4 = [0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1]


In [56]:
# on produit ztest et on teste si c'est égale au résultat voulu
ztest = A5_2_productionv_2(N, R1, R2, R3, R4, f1, f2, f3, f4)

In [57]:
# Maintenant on calcul les autres Registres qui produisent les deux autres suites chiffrantes.
# On transforme les 4 registres trouvés en vecteur pour pouvoir calculer les autres registres.
X1 = vector(R1)
X2 = vector(R2)
X3 = vector(R3)
X4 = vector(R4)

In [58]:
# on calcul les registres qui produisent z1
X1prime = X1 + vector([0 for _ in range(18)]+[IVprime[21]])
X2prime = X2 + vector([0 for _ in range(21)]+[IVprime[21]])
X3prime = X3 + vector([0 for _ in range(22)]+[IVprime[21]])
X4prime = X4 + vector([0 for _ in range(16)]+[IVprime[21]])

# on met un seul bit à 1 dans chaque registre :
X1prime[3] = 1
X2prime[5] = 1
X3prime[4] = 1
X4prime[6] = 1 

# On le remet en sequence
R1prime = Sequence(X1prime)
R2prime = Sequence(X2prime)
R3prime = Sequence(X3prime)
R4prime = Sequence(X4prime)

print("R1prime =",R1prime)
print("R2prime =",R2prime)
print("R3prime =",R3prime)
print("R4prime =",R4prime)

R1prime = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1]
R2prime = [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
R3prime = [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]
R4prime = [0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0]


In [59]:
# et celles qui produisentz2:
X1tilde = X1 + A1 * vector([0 for _ in range(18)]+[IVtilde[20]])
X2tilde = X2 + A2 * vector([0 for _ in range(21)]+[IVtilde[20]])
X3tilde = X3 + A3 * vector([0 for _ in range(22)]+[IVtilde[20]])
X4tilde = X4 + A4 * vector([0 for _ in range(16)]+[IVtilde[20]])

# on met un seul bit à 1 dans chaque registre :
X1tilde[3] = 1
X2tilde[5] = 1
X3tilde[4] = 1
X4tilde[6] = 1 

# On le remet en sequence
R1tilde = Sequence(X1tilde)
R2tilde = Sequence(X2tilde)
R3tilde = Sequence(X3tilde)
R4tilde = Sequence(X4tilde)

print("R1tilde =",R1tilde)
print("R2tilde =",R2tilde)
print("R3tilde =",R3tilde)
print("R4tilde =",R4tilde)

R1tilde = [1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
R2tilde = [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0]
R3tilde = [1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
R4tilde = [0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1]


In [60]:
# On teste les deux suites chiffrantes avec leurs valeurs voulues
zprimetest = A5_2_productionv_2(N, R1prime, R2prime, R3prime, R4prime, f1, f2, f3, f4)
zprimetest == z1

True

In [61]:
ztildetest = A5_2_productionv_2(N, R1tilde, R2tilde, R3tilde, R4tilde, f1, f2, f3, f4)
ztildetest == z2

True

In [62]:
#On voit bien qu on a les bons registres.
# Maintenant on cherche la clef K 
# Pour déclarer les 64 inconnues que l'on utilisera dans la clef K.
BPR = BooleanPolynomialRing(64, 'K')
v = BPR.gens()

# Comme dans l'algorithme de A5/2-init les registres sont vides des 4 LFSR sont nulles.
X1 = vector([0 for _ in range(19)]) # registre du LFSR1
X2 = vector([0 for _ in range(22)]) # registre du LFSR2
X3 = vector([0 for _ in range(23)]) # registre du LFSR3
X4 = vector([0 for _ in range(17)]) # registre du LFSR4

# On calcule les registres comme dans le résultat du (9).
# Dans la première boucle for:
for i in range(64):
    X1 = A1*X1+vector([0 for _ in range(18)]+[v[i]])
    X2 = A2*X2+vector([0 for _ in range(21)]+[v[i]])
    X3 = A3*X3+vector([0 for _ in range(22)]+[v[i]])
    X4 = A4*X4+vector([0 for _ in range(16)]+[v[i]])
    
# Comme IV est nulle, dans la deuxième boucle for on multiplie pour chaque registre par leurs matrices 
# de rétroaction qui sont mis à la puissance 22.
X1 = A1**22*X1
X2 = A2**22*X2
X3 = A3**22*X3
X4 = A4**22*X4

# A la fin de cet algorithme, on met un seul bit à 1 dans chaque registre :
X1[3] = 1
X2[5] = 1
X3[4] = 1
X4[6] = 1           

In [63]:
# On fait une liste équation
L = list(X1)+list(X2)+list(X3)+list(X4)

In [64]:
# On pose l = len(L)
l =len(L)

In [65]:
# On concatène les registres trouvés dans le (8)
R = list(R1)+list(R2)+list(R3)+list(R4)

In [66]:
# On va calculer l'idéal:
I  = ideal([L[i] + R[i] for i in range(81)]);I 

Ideal (K10 + K13 + K16 + K17 + K19 + K20 + K21 + K22 + K25 + K29 + K31 + K33 + K39 + K48 + K49 + K50 + K53 + 1, K0 + K11 + K14 + K17 + K18 + K20 + K21 + K22 + K23 + K26 + K30 + K32 + K34 + K40 + K49 + K50 + K51 + K54, K1 + K12 + K15 + K18 + K19 + K21 + K22 + K23 + K24 + K27 + K31 + K33 + K35 + K41 + K50 + K51 + K52 + K55, 0, K1 + K3 + K14 + K17 + K20 + K21 + K23 + K24 + K25 + K26 + K29 + K33 + K35 + K37 + K43 + K52 + K53 + K54 + K57, K0 + K2 + K4 + K15 + K18 + K21 + K22 + K24 + K25 + K26 + K27 + K30 + K34 + K36 + K38 + K44 + K53 + K54 + K55 + K58, K0 + K1 + K3 + K5 + K16 + K19 + K22 + K23 + K25 + K26 + K27 + K28 + K31 + K35 + K37 + K39 + K45 + K54 + K55 + K56 + K59 + 1, K0 + K1 + K2 + K4 + K6 + K17 + K20 + K23 + K24 + K26 + K27 + K28 + K29 + K32 + K36 + K38 + K40 + K46 + K55 + K56 + K57 + K60 + 1, K0 + K1 + K2 + K3 + K5 + K7 + K18 + K21 + K24 + K25 + K27 + K28 + K29 + K30 + K33 + K37 + K39 + K41 + K47 + K56 + K57 + K58 + K61 + 1, K0 + K1 + K2 + K3 + K4 + K6 + K8 + K19 + K22 + K25 + K26

In [67]:
# On utilise la base groebner.
Iq = I.groebner_basis();list(Iq)

[K0 + 1,
 K1 + 1,
 K2,
 K3,
 K4 + 1,
 K5,
 K6 + 1,
 K7 + 1,
 K8,
 K9 + 1,
 K10,
 K11,
 K12 + 1,
 K13 + 1,
 K14 + 1,
 K15 + 1,
 K16,
 K17 + 1,
 K18,
 K19,
 K20,
 K21,
 K22 + 1,
 K23 + 1,
 K24,
 K25,
 K26,
 K27,
 K28 + 1,
 K29 + 1,
 K30 + 1,
 K31 + 1,
 K32,
 K33 + 1,
 K34,
 K35 + 1,
 K36,
 K37 + 1,
 K38,
 K39,
 K40,
 K41 + 1,
 K42 + 1,
 K43 + 1,
 K44 + 1,
 K45,
 K46 + 1,
 K47 + 1,
 K48,
 K49,
 K50,
 K51 + 1,
 K52 + 1,
 K53 + 1,
 K54 + 1,
 K55 + 1,
 K56,
 K57,
 K58 + 1,
 K59 + 1,
 K60 + 1,
 K61,
 K62,
 K63]

In [68]:
# On cherche les coefficients qui additionnent les variables pour avoir une clef Kprime.
Kprime = [el.constant_coefficient() for el in Iq];Kprime

[1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0]

In [69]:
# On produit avec la clef et leurs IV respectifs
ztest1 = A5_2v_2(N,Kprime, IV)
zprimetest1 = A5_2v_2(N,Kprime, IVprime)
ztildetest = A5_2v_2(N,Kprime, IVtilde)

In [70]:
# on teste avec leurs valeurs voulus
print("ztest1 == z0 is %s" %(ztest1 == z0))
print("zprimetest1 == z1 is %s" %(zprimetest1 == z1))
print("ztildetest == z2 is %s" %(ztildetest == z2))

ztest1 == z0 is True
zprimetest1 == z1 is True
ztildetest == z2 is True
